In [ ]:
%load_ext autoreload
%autoreload 2

# Experimental demonstration of memory-enhanced quantum communication

This notebook follows work published in [This paper](https://arxiv.org/abs/1909.01323) (PDF in current folder). 

### Just enough quantum key distribution (if you need it)

The paper shows a method to improve quantum key distribution using a quantum memory and optical qubits. So we begin with a minimal introduction/reminder on quantum key distributions. The distribution protocol shown below is called BB84 and is **not the one used in the paper**, but it's a basic protocol and relatively simple to explain. It uses optical qubits encoded on the polarization state of photons, but there are many other methods. 

The goal of the game is to exchange a _classical_ encryption key between Alice and Bob, but they want to use a _quantum_ channel to ensure their key was not compromised by the evil Eve. To do this, they agree on two possible orthogonal basis in which they may encode qubits: X (45/135 degree polarization) or + (0/90 degree polarization). Alice randomizes a string of classical bits and polarization orientations and sends them to Bob. Bob chooses an orientation for his polarizer at random and measures the qubit. Alice and Bob then have a chat over an insecure line and Bob tells Alice what polarizer orientations he used for each measurement. For each of Bob's attempts, Alice replies "correct" or "incorrect". The measurements Bob performed with correct orientations resulted in the same bits as those transmitted by Alice and can now constitue an encription key. This is called _sifting_ because you throw away some of the bits (where the polarizer orientation did not match). There are more details on the protocol but they aren't important for us right now.

<center><img src="figure_0.png"/></center>



## The SiV setup


The system under consideration is a Si-V (silicon-vacancy center) in a diamon photonic cavity that is coupled to an optical fiber. This system is used to demonstrate an enhancement in the rate of quantum key distribution (QKD) which uses the SiV as a quantum memory. The memory enables surpassing the theoretical limit of the QKD rate limit in a lossy channel. 


<center><img src="figure_1.png"/></center>


The spin state of the  SiV (up or down) can be controlled using microwave pulses applied by the gold element surrounding the SiV. 

The SiV is embedded in a photonic crystal waveguide which increases the coupling between the incoming photons and the SiV. Cooperativity of this cavity is measured to be  $C=105\pm 11$.

To detect the SiV state, the reflection of a photon from the cavity is measured. If the spin state is $\uparrow$, there is reflection and if it is $\downarrow$, there is none (very little). 

### The photonic qubit

The qubit is encoded on relative phase and timing of a pair of optical pulses, separated by $\delta t = 142 ns$. The pulses are called "early" or "late" photons (though they are weak coherent pulses). 

### Entangling a photonic qubit with a spin 

A photon is refelected from the cavity if and only if the spin state is $\uparrow$. As is shown on the diagram below, the spin is prepared in the $|\leftarrow \rangle= |\uparrow\rangle+ |\downarrow\rangle$ by a $\pi/2$ (MW) pulse and a (MW) $\pi$ pulse is inserted between each pair of photonic pulses. The reflected state can then not have the $|e\uparrow\rangle$ or $| l\downarrow \rangle$ components.

However, as the pulses travel from the cavity and to the pair of single photon detectors, they are passed thorugh a Time-Delay Interferometer (TDI). The interferometer allows photons to go though either a long path or a short path, whose path difference exactly compensates for the $\delta t$ time difference between the pulses. This erases the time-bin information (early or late) and leves the system in an entangled state $|\pm\rangle \propto |e\rangle \pm |l\rangle$ 


### The memory enhanced QKD protocol

We now finally reach the suggested protocol. Unlike the BB84 case, Alice and Bob would be sending qubits to Charlie who's in the middle between them. Charlie holds the memory device (spin) and is able to measure it. However, in this paper the authors do not implement the full protocol. The intent is just to show that a Bell State Measurement (BSM) of photons arriving at different times (i.e. an Asynchronous BSM) owing to the memory life time. So we don't have Alice or Bob, that's why in the system diagram the incoming photons are labeled A/B, indicating it could have been either of them. 

In the image below a set of pulses is shown (red) and for different relative phases (in blue) between each pair (controlled by a optical phase shifter) the intended qubit state is specified below the image. The relative phase between pulses is equivalent to selection of the polarizer direction in the BB84 case. The key point to understand is that on average there are **much less than 1 photon per pulse**. Typically there are something like 0.02 photons per pulse. This means that events where two photons are present in adgacent time bins (comprising a qubit with late and early photons) are even more rare. The experiment post selects for two photon arrival times within a spin cohrence time.

<center><img src="figure_qubits.png"/></center>

### Measuring qubtis in the X basis

As already mentioned a few times photons are reflected from the cavity if and only if the spin in in the $\uparrow$ state. The superposition of early and late photons going through the time delay interferometer can go to either the + or - detectors. If the early photon goes throgh the short path and the late photon goes through the long path, they will be even further separated in time and the recorded time arrival events will be distinguishable. This constitues a measurement in the Z basis. However, if the early photon goes through the short path and the late photon goes through the long path, there will be an arrival event recorded at t=0 (relative the expected photon arrival time in that time step). This is a measurement in the X basis,  interpreted as an early+late or early-late state depending on whether the event was recorded in detector + or -. The image below shows a arrival time histogram on one of the single photon detectors (+1). So the +x state would be measured by any click on the +1 detector at time 0 (relative the photon pair timing) and the -x state would be detected by the same pulse on the -1 detector. 

<center><img src="figure_2.png"/></center>


### Parity measurement

To perfrom the asynchromous BSM, two detector arrival events are needed. They are labeled $m_1$ and $m_2$, each of which gets a value of $\pm 1$ based on the detector to which they arrive. After the weak (red) optical pulses, an additional strong pulse is used to read out the state of the spin. This is given the value $m_3$. The total parity $m=m_1 m_2 m_3$ indicates whether the BSM resulted in a positive or negative parity state. In principle, Charlie communicates the result of the measurement to Alice and Bob. 

At the experimental parameters reported in the paper a "successful" measurement is quite rare, with a rate of about 0.1 Hz. However, this is still a higher rate than would be acheived without the memory as photons from Alice and Bob would have to be prefectly synchronized and simultaneously not be lost due to attenuation in the optical link. 

# QKD IN QUA

Now that we understnad the experiment, we will implement parts of it in QUA. We will not implement the full protocol, but that is a simple extension of the parts we show and is left as an excercise to the reader. 

# Experimental sequence: spin qubit spectroscopy

This first script implements the measurement needed to measure panel (b) in figure 2 of the paper. 
<center><img src="spectroscopy.png"/></center>

The spin qubit is prepared in a superposition state by play a saturation pulse to the MW antenna. This is done assuming that at this stage the $\pi$ pulse time is still unknown and we just want to make sure we have _some_ reflection to work with. A long (30 $\mu$ S) optical pulse is then played and the arrival times of photons to either of the single photon detectors is recorded using the time-tagging operator. 

At each experimental step the frequency of the MW pulse is changed and the experiment is repeated for averaging. 

In [ ]:
from qm.QuantumMachinesManager import QuantumMachinesManager
from qm.qua import *
from qm import SimulationConfig
from Configuration import config

import matplotlib.pyplot as plt

#############
# Time trace:
#############

meas_len = 1e3
rep_num = 1e7
Qmm = QuantumMachinesManager()
Qmm.close_all_quantum_machines()
qm = Qmm.open_qm(config)
with program() as spin_qubit_spec:

    ##############################
    # Declare real-time variables:
    ##############################

    n = declare(int)
    f = declare(int)
    result1 = declare(int, size=int(meas_len/500))
    resultLen1 = declare(int)
    result2 = declare(int, size=int(meas_len/500))
    resultLen2 = declare(int)
    counts = declare(int)


    ###############
    # The sequence:
    ###############

    with for_(n, 0, n < rep_num, n + 1):
        with for_(f, 4e6, f<14e6, f+4e6):
            update_frequency('spin_qubit', f)
            play('saturation', 'spin_qubit', duration=1000)
            align('spin_qubit', 'readout', 'readout1', 'readout2')
            play('on', 'readout', duration=1000)
            measure('readout', 'readout1', 'adc', time_tagging.analog(result1, meas_len, targetLen=resultLen1))
            measure('readout', 'readout2', None, time_tagging.analog(result2, meas_len, targetLen=resultLen2))
            assign(counts, resultLen1+resultLen2)

        save(counts, 'counts')


        # with for_(n, 0, n < resultLen1, n + 1):
        #     save(result1[n], "res1")
        # with for_(p, 0, p < resultLen2, p + 1):
        #     save(result2[p], "res2")


job = qm.simulate(spin_qubit_spec, SimulationConfig(8000))
# job.get_simulated_samples().con1.plot()

sim = job.get_simulated_samples()
pulses = job.simulated_analog_waveforms()
dig_pulses = job.simulated_digital_waveforms()


plt.figure(1)
ax1 = plt.subplot(311)
plt.plot(sim.con1.analog['1'])
plt.plot(sim.con1.analog['2'])
plt.ylabel('spin_qubit')

plt.subplot(312)
plt.plot(sim.con1.digital['1'])
plt.ylabel('readout')

plt.subplot(313)
plt.plot(sim.con1.digital['7'])
plt.plot(sim.con1.digital['8'])
plt.ylabel('counting')


# Experimental sequence: Spin-photon entanglement

In this script, panel (d) from figure 2 is set up. It is a procedure to entangle the photonic qubit and spin which matches the description in "Entangling a photonic qubit with a spin" above. 

<center><img src="spin_photon_entanglement.png"/></center>


Note the usage of _macros_. These are python function where QUA functions are inserted. They allow to write QUA code more concisely and can be very powerful. This technique, of programming one language using another, is called "meta-programming". 

In [ ]:
from qm.QuantumMachinesManager import QuantumMachinesManager
from qm.qua import *
from qm import SimulationConfig
from Configuration import config

import matplotlib.pyplot as plt


#############
# Time trace:
#############
from qua_macros import reset_spin

meas_len = 1e3
rep_num = 1e7
threshold = 7
Qmm = QuantumMachinesManager()
Qmm.close_all_quantum_machines()
qm = Qmm.open_qm(config)
with program() as spin_qubit_spec:

    ##############################
    # Declare real-time variables:
    ##############################

    n = declare(int)
    k = declare(int)
    se = declare(bool)

    result1 = declare(int, size=int(meas_len / 500))
    resultLen1 = declare(int)
    result2 = declare(int, size=int(meas_len / 500))
    resultLen2 = declare(int)

    ###############
    # The sequence:
    ###############
    with for_(n, 0, n < rep_num, n + 1):

        reset_spin(threshold)

        # sequence:
        play('pi2', 'spin_qubit')
        align('spin_qubit', 'opt_qubit_amp')

        play('photon', 'opt_qubit_amp')
        wait(32, 'opt_qubit_amp')
        align('opt_qubit_amp', 'opt_qubit_phase')
        play('phase_shift', 'opt_qubit_phase')
        play('photon', 'opt_qubit_amp')

        wait(18, 'spin_qubit')
        play('pi', 'spin_qubit')
        # align('spin_qubit', 'opt_qubit_amp', 'opt_qubit_phase')



        # Readout:
        align('opt_qubit_amp', 'opt_qubit_phase', 'readout', 'readout1', 'readout2')
        play('on', 'readout', duration=1000)
        measure('readout', 'readout1', None, time_tagging.analog(result1, meas_len, targetLen=resultLen1))
        measure('readout', 'readout2', None, time_tagging.analog(result2, meas_len, targetLen=resultLen2))
        # measure_spin('z', threshold, se)

        # save
        i = declare(int)
        j = declare(int)
        with for_(i, 0, i < resultLen1, i + 1):
            save(result1[i], 'time_tags_plus')
        with for_(j, 0, j < resultLen2, j + 1):
            save(result2[j], 'time_tags_plus')

job = qm.simulate(spin_qubit_spec, SimulationConfig(8000))
# job.get_simulated_samples().con1.plot()

sim = job.get_simulated_samples()
pulses = job.simulated_analog_waveforms()
dig_pulses = job.simulated_digital_waveforms()


plt.figure(1)
ax1 = plt.subplot(411)
plt.plot(sim.con1.analog['1'])
plt.plot(sim.con1.analog['2'])
plt.ylabel('spin_qubit')

plt.subplot(412)
plt.plot(sim.con1.analog['3'])
plt.plot(sim.con1.analog['4'])
plt.plot(sim.con1.digital['3'])
plt.ylabel('opt qubit amp and phase')

plt.subplot(413)
plt.plot(sim.con1.digital['1'])
plt.ylabel('readout')

plt.subplot(414)
plt.plot(sim.con1.digital['7'])
# plt.plot(sim.con1.digital['8'])
plt.ylabel('counting')

